In [1]:
%matplotlib inline
import pandas as pd
import random
import re
from IPython.display import clear_output
import itertools
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

from sklearn.externals import joblib 

In [2]:
train1M = pd.read_csv('train1M.csv',index_col = 0)

In [3]:
train1M.head()

,label,integer_1,integer_2,integer_3,integer_4,integer_5,integer_6,integer_7,integer_8,integer_9,...,categorical_18,categorical_19,categorical_20,categorical_21,categorical_22,categorical_23,categorical_24,categorical_25,categorical_26,Index
19,0,7.0,102,NaN,3.0,780.0,15.0,7.0,15.0,15.0,...,5edd90de,NaN,NaN,e12ce348,NaN,c3dc6cef,49045073,NaN,NaN,19
135,0,NaN,0,17.0,3.0,19811.0,NaN,0.0,3.0,54.0,...,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,32c7478e,43f13e8b,ea9a246c,731c3655,135
177,0,NaN,1,5.0,4.0,2931.0,36.0,2.0,6.0,62.0,...,0f4a15b0,NaN,NaN,0014c32a,NaN,3a171ecb,3b183c5c,NaN,NaN,177
250,0,NaN,0,NaN,0.0,40698.0,963.0,0.0,2.0,23.0,...,281769c2,NaN,NaN,d4703ebd,NaN,32c7478e,aee52b6f,NaN,NaN,250
329,1,NaN,23,83.0,2.0,NaN,NaN,0.0,2.0,2.0,...,f54016b9,21ddcdc9,5840adea,ff3ce4c0,c9d4222a,be7c41b4,d691765a,e8b83407,d1d45fc5,329


# 2.2 Histograms and Stats

In [ ]:
def generateSummaryStatsAndHists(train1M):
    SummaryStats = pd.DataFrame()
    for col in train1M.columns:
        if (col != 'label' and col != 'Index' and col != 'Unnamed: 0'):

            train1M[col][train1M['label'] == 0].value_counts().plot(kind='hist',title=col, bins=100,label='0s')
            train1M[col][train1M['label'] == 1].value_counts().plot(kind='hist',title=col, bins=100,label='1s')
            plt.legend(loc='upper right')
            plt.savefig(col)
            plt.show()
            plt.gcf().clear()
            if (train1M[col].dtype != 'O'):
                SummaryStats[col] = train1M[col].describe()   
    # SummaryStats.head()
    SummaryStats.to_csv('integerStats.csv')
    return SummaryStats

In [ ]:
SummaryStats = generateSummaryStatsAndHists(train1M)
SummaryStats

In [ ]:
train1M['integer_1'][train1M['label'] == 0].value_counts().plot(kind='hist',title='integer_1', bins=100,label='0s')
train1M['integer_1'][train1M['label'] == 1].value_counts().plot(kind='hist',title='integer_1', bins=100,label='1s')
plt.legend(loc='upper right')
plt.show()

# 2.3 Normalization and Feature Engineering

In [4]:
# train1M.head()

In [5]:
# train1M[train1M[train1M.columns[1:14]] == -1]

In [6]:
# Keeping the following categorical features:
# - 9 Because it has 3 distinct peaks
# - 6 because both peaks and >9 points

# Dropping the following categorical features because 
# - Categorical_1 b/c they all have only 1 column for most features 
# - 3
# - 4 
# - 8
# - 10
# - 12
# - 16
# - 19
# - 21
# - 24
# - 26



In [7]:
# encoder = OneHotEncoder()
# encoder.fit_transform(train1M[train1M.columns[14]])
train1M[train1M.columns[14]].dtype

def generateCategoricalData(train1M):
    #change to categorical
    for col in train1M.columns[14:40]:
        train1M[col] = train1M[col].astype('category')
        # add the dummy category
        train1M[col].cat.add_categories(new_categories = 'Dummy',inplace = True)
        categories = pd.Series(train1M[col].cat.categories)
        categories.to_csv(str(col)+'_features.csv',header = False)
        #save the categories for each column
        #then we can set the categegories for each column
        # and when we get dummies from pandas we have a one hot encoding that is consistent accross
        # -> get_dummies() method does one hot encoding
        
generateCategoricalData(train1M)

In [8]:
def preProcessData(train1M):
    train1M[train1M.columns[1:14]] = train1M[train1M.columns[1:14]].fillna(0)
    train1M[train1M.columns[14:40]] = train1M[train1M.columns[14:40]].fillna('Dummy')
    train1M[train1M.columns[1:14]] = train1M[train1M.columns[1:14]].replace(-1,0)

def dropFeature(series,feature):
    
    for index, value in series.iteritems():
        if (value == feature):
#             print('index: ', index, 'value: ', value)
            series = series.drop(index)
            return series            
    return -1

def preProcessInts(train1M):
    # mean-center and unit variance the integers
    trainingMeanSTD = pd.read_csv('training_stats_integers.csv',index_col=0)
    print(trainingMeanSTD.head())
#     for col in train1M.columns: 
        
        
    
    train1M[train1M.columns[1:14]] = scale(train1M[train1M.columns[1:14]])

preProcessData(train1M)

preProcessInts(train1M)


featuresToKeep = pd.Series(data = train1M.columns)
# print(featuresToKeep)
featuresToKeep = dropFeature(featuresToKeep,'Index')
featuresToKeep = dropFeature(featuresToKeep,'label')
featuresToKeep = dropFeature(featuresToKeep,'categorical_1')
featuresToKeep = dropFeature(featuresToKeep,'categorical_3')
featuresToKeep = dropFeature(featuresToKeep,'categorical_4')
featuresToKeep = dropFeature(featuresToKeep,'categorical_5')
featuresToKeep = dropFeature(featuresToKeep,'categorical_8')
featuresToKeep = dropFeature(featuresToKeep,'categorical_10')
featuresToKeep = dropFeature(featuresToKeep,'categorical_12')
featuresToKeep = dropFeature(featuresToKeep,'categorical_16')
featuresToKeep = dropFeature(featuresToKeep,'categorical_19')
featuresToKeep = dropFeature(featuresToKeep,'categorical_21')
featuresToKeep = dropFeature(featuresToKeep,'categorical_24')
featuresToKeep = dropFeature(featuresToKeep,'categorical_26')

featuresToKeep.to_csv('features.txt',header=False, index=False)

         integer_1     integer_2     integer_3     integer_4     integer_5  \
mean -9.846092e-15  1.489631e-15 -4.199304e-15  7.936985e-15 -5.790918e-17   
std   1.000000e+00  1.000001e+00  1.000000e+00  1.000000e+00  1.000001e+00   

         integer_6     integer_7     integer_8     integer_9    integer_10  \
mean -4.077688e-17 -1.721416e-14 -1.276412e-16  7.152975e-15  1.552617e-14   
std   1.000001e+00  1.000001e+00  1.000001e+00  1.000001e+00  1.000001e+00   

        integer_11    integer_12    integer_13  
mean -1.178304e-13  1.435046e-14 -1.068410e-14  
std   1.000001e+00  1.000001e+00  1.000000e+00  


In [9]:
#Now that I have my kept features:

In [10]:
# #generate the list of features and assign an integer value to them (the index is the integer value)
# for col in train1M.columns[14:40]:
#     print(train1M[col].cat.categories)
#     print(train1M[col].cat.codes)
#     pd.Series(train1M[col].cat.categories, index=train1M[col].cat.codes).to_csv(str(col)+'_features.csv',header = False)

In [12]:
for col in train1M.columns[14:40]:
    # load the current Feature spaces
    curFeatures = pd.read_csv(str(col) + "_features.csv",header = None,index_col = 0)
    
    train1M = pd.concat([train1M, pd.get_dummies(train1M[col],prefix=['encoded'],sparse=True)])
    
    print("done " + col)

done categorical_1
done categorical_2


KeyboardInterrupt: 

In [15]:
train1M.columns

Index(['label', 'integer_1', 'integer_2', 'integer_3', 'integer_4',
       'integer_5', 'integer_6', 'integer_7', 'integer_8', 'integer_9',
       'integer_10', 'integer_11', 'integer_12', 'integer_13', 'categorical_1',
       'categorical_2', 'categorical_3', 'categorical_4', 'categorical_5',
       'categorical_6', 'categorical_7', 'categorical_8', 'categorical_9',
       'categorical_10', 'categorical_11', 'categorical_12', 'categorical_13',
       'categorical_14', 'categorical_15', 'categorical_16', 'categorical_17',
       'categorical_18', 'categorical_19', 'categorical_20', 'categorical_21',
       'categorical_22', 'categorical_23', 'categorical_24', 'categorical_25',
       'categorical_26', 'Index'],
      dtype='object')

In [ ]:
train1M['categorical_1']

In [53]:
def preProcessIntsAndSave(train1M,fileName):
    # mean-center and unit variance the integers
#     trainingMeanSTD = pd.read_csv('training_stats_integers.csv',index_col=0)
#     print(trainingMeanSTD.head())
    curScaler = StandardScaler()
    curScaler.fit(train1M[train1M.columns[1:14]])
    joblib.dump(curScaler, fileName) 

In [54]:
def getMeanAndSTD(series):

    return pd.DataFrame(data = [series.mean(), series.std()], index = ['mean','std'])

getMeanAndSTD(train1M[train1M.columns[1:14]]).to_csv('training_stats_integers.csv')

In [55]:
preProcessIntsAndSave(train1M[train1M.columns[1:14]],'scalerPickle')

[[-0.01140292 -0.0595101  -0.32714324 ... -0.31593225 -0.09583118
  -0.24232319]
 [-0.27098801 -0.01190302 -0.32714324 ... -0.51150991 -0.09583118
  -0.24232319]
 [-0.26844306 -0.04550802 -0.207158   ... -0.31593225 -0.09583118
  -0.1005595 ]
 ...
 [-0.27098801 -0.05670968 -0.44712848 ...  1.0531114  -0.09583118
  -0.31320503]
 [ 0.28381149 -0.03990718 -0.44712848 ... -0.12035458 -0.09583118
  -0.31320503]
 [ 1.86422658 -0.05390927 -0.68709897 ... -0.51150991 -0.09583118
  -0.45496872]]


In [ ]:
scaler = joblib.load('scalerPickle') 

In [ ]:

scaler.transform(train1M[train1M.columns[1:14]])

In [ ]:
train1M[train1M.columns[1:14]]